In [129]:
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import AllTimeLeadersGrids

In [130]:
commonallplayers.CommonAllPlayers(is_only_current_season=1)

In [60]:
import sys

In [135]:
all_players = commonallplayers.CommonAllPlayers(is_only_current_season=0).get_data_frames()[0]

In [191]:
from nba_api.stats.endpoints import commonplayerinfo
commonplayerinfo.CommonPlayerInfo(player_id="203084").get_data_frames()[0]

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,ROSTERSTATUS,GAMES_PLAYED_CURRENT_SEASON_FLAG,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_CITY,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
0,203084,Harrison,Barnes,Harrison Barnes,"Barnes, Harrison",H. Barnes,harrison-barnes,1992-05-30T00:00:00,North Carolina,USA,North Carolina/USA,6-8,225,11,40,Forward,Active,Y,1610612758,Kings,SAC,kings,Sacramento,harrison_barnes,2012,2023,N,Y,Y,2012,1,7,N


In [203]:
from nba_api.stats.endpoints import winprobabilitypbp
winprobabilitypbp.WinProbabilityPBP(game_id = "0022300827").get_data_frames()[0]

,GAME_ID,EVENT_NUM,HOME_PCT,VISITOR_PCT,HOME_PTS,VISITOR_PTS,HOME_SCORE_MARGIN,PERIOD,SECONDS_REMAINING,HOME_POSS_IND,HOME_G,DESCRIPTION,LOCATION,PCTIMESTRING,ISVISIBLE
0,0022300827,2.0,0.58808,0.41192,0,0,0,1,720.0,NaN,None,Start of 1st Period (9:47 PM EST),,12:00,1.0
1,0022300827,4.0,0.61352,0.38648,0,0,0,1,720.0,1.0,None,Jump Ball Zubac vs. Sabonis: Tip to Harden,h,12:00,1.0
2,0022300827,NaN,0.61350,0.38650,0,0,0,1,719.0,1.0,None,None,None,None,NaN
3,0022300827,NaN,0.61348,0.38652,0,0,0,1,718.0,1.0,None,None,None,None,NaN
4,0022300827,NaN,0.61346,0.38654,0,0,0,1,717.0,1.0,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3011,0022300827,NaN,NaN,NaN,107,123,-16,4,4.0,0.0,None,None,None,None,NaN
3012,0022300827,NaN,NaN,NaN,107,123,-16,4,3.0,0.0,None,None,None,None,NaN
3013,0022300827,NaN,NaN,NaN,107,123,-16,4,2.0,0.0,None,None,None,None,NaN
3014,0022300827,NaN,NaN,NaN,107,123,-16,4,1.0,0.0,None,None,None,None,NaN


In [202]:
from nba_api.stats.endpoints import CumeStatsPlayerGames
CumeStatsPlayerGames(player_id="203084").get_data_frames()[0]

,MATCHUP,GAME_ID
0,02/25/2024 Kings at Clippers,0022300827
1,02/22/2024 Spurs at Kings,0022300803
2,02/14/2024 Kings at Nuggets,0022300786
3,02/13/2024 Kings at Suns,0022300774
4,02/11/2024 Kings at Thunder,0022300760
5,02/09/2024 Nuggets at Kings,0022300746
6,02/07/2024 Pistons at Kings,0022300732
7,02/05/2024 Kings at Cavaliers,0022300714
8,02/03/2024 Kings at Bulls,0022300700
9,02/02/2024 Kings at Pacers,0022300691


In [185]:
from nba_api.stats.endpoints import CommonTeamRoster
# This is good
CommonTeamRoster(team_id="1610612758", season="2020-21").get_data_frames()[0]

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612758,2020,00,Tyrese Haliburton,Tyrese,tyrese-haliburton,0,G,6-5,185,"FEB 29, 2000",21.0,R,Iowa State,1630169,None
1,1610612758,2020,00,Jahmi'us Ramsey,Jahmi'us,jahmius-ramsey,3,G,6-3,190,"JUN 09, 2001",20.0,R,Texas Tech,1630186,None
2,1610612758,2020,00,De'Aaron Fox,De'Aaron,deaaron-fox,5,G,6-3,185,"DEC 20, 1997",23.0,3,Kentucky,1628368,#5 Pick in 2017 Draft
3,1610612758,2020,00,Kyle Guy,Kyle,kyle-guy,7,G,6-1,167,"AUG 11, 1997",23.0,1,Virginia,1629657,None
4,1610612758,2020,00,Maurice Harkless,Maurice,maurice-harkless,8,F-G,6-7,220,"MAY 11, 1993",28.0,8,"St. John's, N.Y.",203090,None
5,1610612758,2020,00,Terence Davis,Terence,terence-davis,9,G,6-4,201,"MAY 16, 1997",24.0,1,Mississippi,1629056,None
6,1610612758,2020,00,Justin James,Justin,justin-james,10,G-F,6-7,190,"JAN 24, 1997",24.0,1,Wyoming,1629713,None
7,1610612758,2020,00,Robert Woodard II,Robert,robert-woodard-ii,13,F,6-6,235,"SEP 22, 1999",21.0,R,Mississippi State,1630218,None
8,1610612758,2020,00,Damian Jones,Damian,damian-jones,15,C,6-11,245,"JUN 30, 1995",26.0,4,Vanderbilt,1627745,None
9,1610612758,2020,00,Hassan Whiteside,Hassan,hassan-whiteside,20,C,7-0,265,"JUN 13, 1989",32.0,8,Marshall,202355,None


In [139]:
all_players

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_SLUG,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
0,76001,"Abdelnaby, Alaa",Alaa Abdelnaby,0,1990,1994,HISTADD_alaa_abdelnaby,alaa_abdelnaby,0,,,,,None,Y,00
1,76002,"Abdul-Aziz, Zaid",Zaid Abdul-Aziz,0,1968,1977,HISTADD_zaid_abdul-aziz,zaid_abdul-aziz,0,,,,,None,Y,00
2,76003,"Abdul-Jabbar, Kareem",Kareem Abdul-Jabbar,0,1969,1988,HISTADD_kareem_abdul-jabbar,kareem_abdul-jabbar,0,,,,,None,Y,00
3,51,"Abdul-Rauf, Mahmoud",Mahmoud Abdul-Rauf,0,1990,2000,mahmoud_abdul-rauf,mahmoud_abdul-rauf,0,,,,,None,Y,00
4,1505,"Abdul-Wahad, Tariq",Tariq Abdul-Wahad,0,1997,2003,tariq_abdul-wahad,tariq_abdul-wahad,0,,,,,None,Y,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4909,1627790,"Zizic, Ante",Ante Zizic,0,2017,2019,ante_zizic,ante_zizic,0,,,,,None,Y,01
4910,78647,"Zoet, Jim",Jim Zoet,0,1982,1982,HISTADD_jim_zoet,jim_zoet,0,,,,,None,Y,00
4911,78648,"Zopf, Bill",Bill Zopf,0,1970,1970,HISTADD_zip_zopf,bill_zopf,0,,,,,None,Y,00
4912,1627826,"Zubac, Ivica",Ivica Zubac,1,2016,2023,ivica_zubac,ivica_zubac,1610612746,LA,Clippers,LAC,clippers,clippers,Y,01


In [186]:
import numpy as np
import pandas as pd

In [187]:
from nba_api.stats.endpoints import teamdetails, teamplayerdashboard, commonallplayers

def get_team_roster(team_id):
    # Get all players who have played for the team
    players_info = commonallplayers.CommonAllPlayers(is_only_current_season=1)
    players_info_data = players_info.get_data_frames()[0]
    
    # Filter players by team ID
    roster = players_info_data[players_info_data["TEAM_ID"] == team_id]
    
    return roster

def get_team_stats(team_id, season):
    # Get team stats for a specific season
    team_stats = teamplayerdashboard.TeamPlayerDashboard(team_id=team_id, season=season)
    team_stats_data = team_stats.get_data_frames()[0]
    
    # Extract team stats
    team_stats = team_stats_data
    
    return team_stats

In [188]:
from nba_api.stats.static import teams

def get_team_ids():
    # Retrieve all NBA teams
    nba_teams = teams.get_teams()
    
    team_ids = {}
    # Extract team IDs
    for team in nba_teams:
        team_ids[team["full_name"]] = team['id']
    
    return team_ids

In [189]:
get_team_ids()

{'Atlanta Hawks': 1610612737,
 'Boston Celtics': 1610612738,
 'Cleveland Cavaliers': 1610612739,
 'New Orleans Pelicans': 1610612740,
 'Chicago Bulls': 1610612741,
 'Dallas Mavericks': 1610612742,
 'Denver Nuggets': 1610612743,
 'Golden State Warriors': 1610612744,
 'Houston Rockets': 1610612745,
 'Los Angeles Clippers': 1610612746,
 'Los Angeles Lakers': 1610612747,
 'Miami Heat': 1610612748,
 'Milwaukee Bucks': 1610612749,
 'Minnesota Timberwolves': 1610612750,
 'Brooklyn Nets': 1610612751,
 'New York Knicks': 1610612752,
 'Orlando Magic': 1610612753,
 'Indiana Pacers': 1610612754,
 'Philadelphia 76ers': 1610612755,
 'Phoenix Suns': 1610612756,
 'Portland Trail Blazers': 1610612757,
 'Sacramento Kings': 1610612758,
 'San Antonio Spurs': 1610612759,
 'Oklahoma City Thunder': 1610612760,
 'Toronto Raptors': 1610612761,
 'Utah Jazz': 1610612762,
 'Memphis Grizzlies': 1610612763,
 'Washington Wizards': 1610612764,
 'Detroit Pistons': 1610612765,
 'Charlotte Hornets': 1610612766}

In [163]:
# Specify team ID and season
team_id = 1610612758  # Atlanta Hawks
season = '2023-24'

# Get team roster
roster = get_team_roster(team_id)
print("Team Roster:")
print(roster[["DISPLAY_FIRST_LAST", "PERSON_ID"]])
    

# Get team stats
team_stats = get_team_stats(team_id, season)
print("\nTeam Stats for Season", season)
team_stats

Team Roster:
    DISPLAY_FIRST_LAST  PERSON_ID
29     Harrison Barnes     203084
135       Chris Duarte    1630537
142    Kessler Edwards    1630556
143         Keon Ellis    1631165
153        Jordan Ford    1630259
156       De'Aaron Fox    1628368
231      Kevin Huerter    1628989
261        Colby Jones    1641732
264        Mason Jones    1630222
291           Alex Len     203458
308         Trey Lyles    1626168
331       JaVale McGee     201580
347    Davion Mitchell    1630558
351         Malik Monk    1628370
363      Keegan Murray    1631099
440   Domantas Sabonis    1627734
463      Jalen Slawson    1641771
510     Sasha Vezenkov    1628426

Team Stats for Season 2023-24


,GROUP_SET,TEAM_ID,TEAM_NAME,GROUP_VALUE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,Overall,1610612758,Sacramento Kings,2023-24,56,33,23,0.589,2713.0,2465,5092,0.484,826,2228,0.371,888,1211,0.733,575,1844,2419,1605,749.0,404,221,269,1116,1048,6644,47.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
